# 🔗 LangChain - Conceito de Chains com Gemini

As **Chains** em LangChain são fluxos de execução que conectam modelos de linguagem com outras ferramentas, memórias e lógicas. Elas permitem criar aplicações mais estruturadas e reutilizáveis.

---

## 🧩 Tipos de Chains com Exemplos Completos

### 1. 💬 ConversationChain
Cadeia simples que conecta um modelo de linguagem com uma memória para manter o contexto da conversa.

```bash
from langchain.chains import ConversationChain
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory

# Inicializa o modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Cria a memória para armazenar o histórico da conversa
memory = ConversationBufferMemory()

# Cria a cadeia de conversa
conversation = ConversationChain(llm=llm, memory=memory)

# Executa uma interação
response = conversation.run("Qual é a capital do Brasil?")
```
### 2. 📄 LLMChain
Executa uma tarefa com base em um prompt personalizado. Ideal para tarefas específicas como geração de texto, resumo, tradução, etc.
```bash
from langchain.chains import LLMChain
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Inicializa o modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Define o prompt com variáveis
prompt = PromptTemplate(
    input_variables=["tema"],
    template="Explique de forma simples o seguinte tema: {tema}"
)

# Cria a cadeia com o prompt e o modelo
chain = LLMChain(llm=llm, prompt=prompt)

# Executa a cadeia com um tema
response = chain.run("Blockchain")


```

### 3. 🧠 SimpleSequentialChain
Executa múltiplas cadeias em sequência, passando a saída de uma como entrada da próxima.
```bash
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Primeira cadeia: gera uma ideia
prompt_1 = PromptTemplate(input_variables=["tema"], template="Crie um título criativo sobre {tema}")
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# Segunda cadeia: desenvolve um parágrafo com base no título
prompt_2 = PromptTemplate(input_variables=["titulo"], template="Escreva um parágrafo sobre: {titulo}")
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

# Cadeia sequencial
sequential_chain = SimpleSequentialChain(chains=[chain_1, chain_2])

# Executa a cadeia
response = sequential_chain.run("inteligência artificial na educação")


```

### 4. 🔄 SequentialChain (com múltiplas variáveis)
Permite controle mais detalhado sobre entradas e saídas entre as etapas.
```bash
from langchain.chains import SequentialChain
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Cadeia 1: gera um título
prompt_1 = PromptTemplate(input_variables=["tema"], template="Crie um título para um artigo sobre {tema}")
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="titulo")

# Cadeia 2: gera um resumo com base no título
prompt_2 = PromptTemplate(input_variables=["titulo"], template="Escreva um resumo para o artigo: {titulo}")
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="resumo")

# Cadeia sequencial com múltiplas variáveis
overall_chain = SequentialChain(
    chains=[chain_1, chain_2],
    input_variables=["tema"],
    output_variables=["titulo", "resumo"]
)

# Executa a cadeia
result = overall_chain.run({"tema": "tecnologia sustentável"})


```

### 5. 🧪 TransformChain (avançado)
Aplica transformações personalizadas nos dados entre etapas. Ideal para pipelines complexos.

```bash
from langchain.chains import TransformChain
from langchain.chat_models import ChatGoogleGenerativeAI

# Função de transformação personalizada
def transformar(inputs):
    texto = inputs["texto"]
    return {"texto_maiusculo": texto.upper()}

# Cria a cadeia de transformação
chain = TransformChain(
    input_variables=["texto"],
    output_variables=["texto_maiusculo"],
    transform=transformar
)

# Executa a transformação
output = chain.run({"texto": "langchain é poderoso"})


```

# 🚦 LangChain - RouterChain com Gemini

A **RouterChain** permite **direcionar dinamicamente** uma entrada para diferentes cadeias (chains), com base em regras, intenção do usuário ou classificação. É ideal para:

- 🧠 Assistentes multifuncionais
- 🛠️ Aplicações com múltiplos fluxos
- 🗂️ Classificação de tarefas

---

## 🧭 Como Funciona

1. O usuário envia uma entrada.
2. Um modelo de linguagem (LLM) analisa a intenção.
3. A entrada é roteada para a cadeia apropriada.
4. A resposta é gerada com base na cadeia selecionada.

---

## 🧪 Exemplo com Gemini

```bash
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatGoogleGenerativeAI

# Inicializa o modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Define prompts específicos para cada tipo de tarefa
math_prompt = PromptTemplate(
    input_variables=["input"],
    template="Resolva o seguinte problema matemático: {input}"
)

code_prompt = PromptTemplate(
    input_variables=["input"],
    template="Escreva um código Python para: {input}"
)

general_prompt = PromptTemplate(
    input_variables=["input"],
    template="Responda de forma geral: {input}"
)

# Cria cadeias específicas
math_chain = LLMChain(llm=llm, prompt=math_prompt)
code_chain = LLMChain(llm=llm, prompt=code_prompt)
general_chain = LLMChain(llm=llm, prompt=general_prompt)

# Define os destinos com descrições
destination_chains = {
    "math": math_chain,
    "code": code_chain,
    "general": general_chain
}

# Prompt para decidir o destino
router_prompt = PromptTemplate(
    input_variables=["input"],
    template="Classifique a seguinte entrada como 'math', 'code' ou 'general': {input}"
)

# Cadeia de roteamento
router_chain = MultiPromptChain(
    llm_chain=LLMChain(llm=llm, prompt=router_prompt),
    destination_chains=destination_chains,
    default_chain=general_chain
)

# Testa o roteador com uma entrada
response = router_chain.run("Calcule a raiz quadrada de 144")




In [ ]:
from langchain_openai.chat_models import ChatOpenAI

from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

In [ ]:
chat = ChatOpenAI(model="gpt-tubo")
memory=ConversationBufferMemory()
chain=ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

In [ ]:
chat.predict(input("olá"))

In [ ]:
from langchain.prompts import PromptTemplate

propmt_templete=PromptTemplate("""
Essa é uma conversa amigavel :
conversa atual: 
{history}
Human:{input}
AI:
""")

In [ ]:
chat= ChatOpenAI()
memory= ConversationBufferMemory()
chain=ConversationChain(
    prompt=propmt_templete,
    llm=chat,
    memory=memory,
    verbose=True
)

In [ ]:
chain.predict(input("ola"))

### LLM Chain

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat=ChatOpenAI(model="gpt-3.5")



In [ ]:
#prompt

prompt= PromptTemplate.from_template(
    """
        Escolha o melhor nome para mim sobre uma empresa que sesenvolve soluçoes em {produto}

    """
)

chain= LLMChain(
    llm=chat,
    prompt=prompt
)

produto="llm com Rag"
chain.run(produto)